In [2]:
!pip install implicit

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [5]:
data = pd.read_csv('../data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [20]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """    
    
    recs = np.random.choice(items_weights['item_id'], size=n, p=items_weights['weight'], replace=False)
    
    return recs.tolist()

In [21]:
%%time

# Нужно каким-то образом определить веса на вход функции.
# Для примера построим веса исходя из продаж по каждому товару.
popular = data.groupby('item_id')['sales_value'].sum().reset_index()
total = popular['sales_value'].sum()
popular['weight'] = popular['sales_value']/total
popular['weight'].sum()

# Выбираем 5 случайных товаров, но с учетом весов.
weighted_random_recommendation(popular, n=5)

CPU times: user 56.3 ms, sys: 70 ms, total: 126 ms
Wall time: 206 ms


[13381928, 6534178, 8159799, 1044078, 1104584]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [79]:
result = pd.read_csv('../data/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [168]:
def precision_at_k(recommended_list, bought_list, k=5):    
      
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)   
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / recommended_list.size    
    
    return precision

In [169]:
result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2037    0.0
2038    0.0
2039    0.0
2040    0.0
2041    0.0
Length: 2042, dtype: float64

In [122]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()

0.0

In [118]:
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()

0.0

In [119]:
result.apply(lambda row: precision_at_k(row['cosine'], row['actual']), axis=1).mean()


0.0

In [120]:
result.apply(lambda row: precision_at_k(row['tfidf'], row['actual']), axis=1).mean()

0.0

In [121]:
result.apply(lambda row: precision_at_k(row['own_purchases'], row['actual']), axis=1).mean()

0.0